# Classification Prediction Model: Client Term Bank Deposit

By: Katrina James, Christian Urday, Justin Kerry

## Introductory Statement

Banks offer a wide variety of financial services to the population and have long found themselves as a critical element in the foundation of our civilization. As a society, we rely on banks as a major pillar for the overall national economic health and stability. As individuals, we rely on banks as institutions that allow us to hold, borrow, and deposit money in a highly organized and archived manner. Due to this, banking is widely considered a service and an ally for many people. Although that is true, banking should also be considered a business. And like any other type of business, there are limitations and restrictions held in place to protect and safeguard the business. Some of these limitations and restrictions can include charging clients a penalty for early withdrawals and maintaining interest rates against rising inflation. What we are studying here is called a term deposit.

## Background

A term deposit is a fixed term investment made by a client when they deposit money into an account. This investment is then taken by the bank and loaned to other clients or invested in other finacial products with a higher rate of return. Since banks can be considered businesses, they ideally want to pay back to the investor the lowest possible rate of interest for the term deposit and generate the highest possible rate of interest through the loan or product of investment. Ultimately term deposits can be attractive for low risk investors since they are risk free however certain restrictions can lead to clients opting out of a term deposit. A term deposit that is made must be held by the bank for a specified time period and not withdrawn earlier or there will be a penalty charged. Interest rates paid to investors do not keep up with the rising level of inflation over time. Many other fixed-rate investments pay higher interest rates than term deposits. So even though term deposits can be attractive to investors because they are a low risk investment, these restrictions can lead to clients choosing not to invest in term deposits.

## Objective

We will be using a machine learning model to predict whether or not a bank client will choose to invest in the low risk yet limited investment known as a term deposit. This can be achieved by utilizing some dependent variables that have a great deal of predictive power while at the same time avoiding multi-collinearity that will hinder the coefficients of our independent variables creating bias in our models.

## Analysis

For our research, we will use a dataset that we sourced from Kaggle called Bank Marketing (Binary Classification). 

This dataset was created by: Paulo Cortez (Univ. Minho) and Sérgio Moro (ISCTE-IUL) @ 2012.
It was fully described and analysed in S. Moro, R. Laureano and P. Cortez. Using Data Mining for Bank Direct Marketing: An Application of the CRISP-DM Methodology.

For the purposes of the assignment we have extracted the full-dataset which includes over 40,000 records. The number of attributes that this dataset contains is 16 outputs.

The following code will generate the dataset as a viewable table. This will allow us to view all variables including the 'deposit' column which as mentioned previously is our target variable:

In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, auc, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
import statsmodels.api as sm

pd.set_option('display.max_columns', None)

bank_df = pd.read_csv(os.path.join('data', 'Bank.csv'), index_col=0)
bank_df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,deposit
0,44,blue-collar,married,basic.4y,unknown,yes,no,cellular,aug,thu,210,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,no
1,53,technician,married,unknown,no,no,no,cellular,nov,fri,138,1,999,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,no
2,28,management,single,university.degree,no,yes,no,cellular,jun,thu,339,3,6,2,success,-1.7,94.055,-39.8,0.729,4991.6,yes
3,39,services,married,high.school,no,no,no,cellular,apr,fri,185,2,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,no
4,55,retired,married,basic.4y,no,yes,no,cellular,aug,fri,137,1,3,1,success,-2.9,92.201,-31.4,0.869,5076.2,yes


## Purpose

The dataset was collected with the intention of predicting whether or not the client of a bank will subscribe to a term deposit. We decided on this dataset because we found that it was clear and concise. It also has sufficient records of data as well as sufficient independent variables that are somewhat relatable to our target variable.

## Representation

For the purposes of our classification prediction models, we can define and measure the outcomes from the dataset using just a couple of different categorical values. These values include people who did subscribe for term deposit encoded as '1' and people who did not subscribe for the deposit encoded as '0'. We will encode these binary values to the dataset using the .map function in python.

In [2]:
bank_df['deposit'] = bank_df['deposit'].map({'yes': 1, 'no': 0})
bank_df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,deposit
0,44,blue-collar,married,basic.4y,unknown,yes,no,cellular,aug,thu,210,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
1,53,technician,married,unknown,no,no,no,cellular,nov,fri,138,1,999,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,0
2,28,management,single,university.degree,no,yes,no,cellular,jun,thu,339,3,6,2,success,-1.7,94.055,-39.8,0.729,4991.6,1
3,39,services,married,high.school,no,no,no,cellular,apr,fri,185,2,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,0
4,55,retired,married,basic.4y,no,yes,no,cellular,aug,fri,137,1,3,1,success,-2.9,92.201,-31.4,0.869,5076.2,1


We had to represent 'was not previously contacted' (pdays = 999) as a boolean value which is now shown in the following table under 'client_was_contacted' (0 for no and 1 for yes). This feature engineering is important since it allows our models to recognize whether the client was contacted or not by the bank.

In [3]:
bank_df['client_was_contacted'] = np.where(bank_df['pdays'] == 999, 0, 1)
bank_df['pdays'] = np.where(bank_df['pdays'] == 999, 0, bank_df['pdays'])
bank_df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,deposit,client_was_contacted
0,44,blue-collar,married,basic.4y,unknown,yes,no,cellular,aug,thu,210,1,0,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0,0
1,53,technician,married,unknown,no,no,no,cellular,nov,fri,138,1,0,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,0,0
2,28,management,single,university.degree,no,yes,no,cellular,jun,thu,339,3,6,2,success,-1.7,94.055,-39.8,0.729,4991.6,1,1
3,39,services,married,high.school,no,no,no,cellular,apr,fri,185,2,0,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,0,0
4,55,retired,married,basic.4y,no,yes,no,cellular,aug,fri,137,1,3,1,success,-2.9,92.201,-31.4,0.869,5076.2,1,1



We will also measure the effectiveness of our algorithms using a Test Confusion Matrix which will show us a test accuracy score likely displayed as a decimal number to provide us with a percentage. Confusion Matrix for both models will be displayed later in this report at appropriate time.

## Constructing a Final Dataset

In order for our classification prediction models to be more accurate, we had to modify some variables in our dataset. First we had to use the info_value_calc function with an if statement in python to model text data as categorical and numerical data as continous. Continuous data has been binned into decimals for normalization:

In [4]:
def info_value_calc(df, column, is_categorical):
    if is_categorical:
        info_val_df = df.groupby([column])['deposit'].agg(['count', 'sum'])
    else:
        df['variable_bin'] = pd.qcut(df[column].rank(method='first'), 10)
        info_val_df = df.groupby(['variable_bin'])['deposit'].agg(['count', 'sum'])
    
    info_val_df = info_val_df.rename(columns={'sum': 'bad'})
    info_val_df["good"] = info_val_df["count"] - info_val_df["bad"]
    info_val_df["bad_percentage"] = info_val_df["bad"] / info_val_df["bad"].sum()
    info_val_df["good_percentage"] = info_val_df["good"] / info_val_df["good"].sum()
    info_val_df["information_value"] = info_val_df.apply(lambda row: (row['good_percentage'] - row['bad_percentage']) * (np.log(row["good_percentage"] / row["bad_percentage"]) if row["bad_percentage"] != 0.0 else 1.0), axis=1)
    return info_val_df

In [5]:
print(info_value_calc(bank_df, 'education', True))
print('Total information value: ' + str(info_value_calc(bank_df, 'education', True)['information_value'].sum()))

                     count   bad   good  bad_percentage  good_percentage  \
education                                                                  
basic.4y              4176   428   3748        0.092241         0.102550   
basic.6y              2292   188   2104        0.040517         0.057568   
basic.9y              6045   473   5572        0.101940         0.152457   
high.school           9515  1031   8484        0.222198         0.232133   
illiterate              18     4     14        0.000862         0.000383   
professional.course   5243   595   4648        0.128233         0.127175   
university.degree    12168  1670  10498        0.359914         0.287239   
unknown               1731   251   1480        0.054095         0.040495   

                     information_value  
education                               
basic.4y                      0.001092  
basic.6y                      0.005989  
basic.9y                      0.020333  
high.school                   0.00

### Predictive Power

Another process we had to consider was removing unnecessary variables that didn't have any predictive power for our models. Before we could do that however we had to address other considerations prior. We first performed an information value calculation to determine predictive power. When we printed out the information value we ended up with the following results:

Now that we have an idea of predictive power, we have a better understanding on which variables can be removed. However before we start removing them, we should manipulate the data further to achieve more specific information regarding our data.

### Dummy Variables

In order for our models to handle categorical data types, we will need to transpose them into dummy variables:

In [6]:
client_contacted_dummies = pd.get_dummies(bank_df['client_was_contacted'], prefix='client_was_contacted')
poutcome_dummies = pd.get_dummies(bank_df['poutcome'], prefix='poutcome')
month_dummies = pd.get_dummies(bank_df['month'], prefix='month')
contact_dummies = pd.get_dummies(bank_df['contact'], prefix='contact')
job_dummies = pd.get_dummies(bank_df['job'], prefix='job')
default_dummies = pd.get_dummies(bank_df['default'], prefix='default')

bank_model_df = pd.concat([bank_df[['duration', 'nr_employed', 'euribor3m', 'emp_var_rate', 'cons_conf_idx', 'cons_price_idx', 'deposit', 
                                   'pdays', 'age', 'previous']],
                          client_contacted_dummies, poutcome_dummies, month_dummies, 
                          contact_dummies, job_dummies, default_dummies], axis=1)
bank_model_df.head()

,duration,nr_employed,euribor3m,emp_var_rate,cons_conf_idx,cons_price_idx,deposit,pdays,age,previous,client_was_contacted_0,client_was_contacted_1,poutcome_failure,poutcome_nonexistent,poutcome_success,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,contact_cellular,contact_telephone,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,default_no,default_unknown,default_yes
0,210,5228.1,4.963,1.4,-36.1,93.444,0,0,44,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
1,138,5195.8,4.021,-0.1,-42.0,93.200,0,0,53,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2,339,4991.6,0.729,-1.7,-39.8,94.055,1,6,28,2,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
3,185,5099.1,1.405,-1.8,-47.1,93.075,0,0,39,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
4,137,5076.2,0.869,-2.9,-31.4,92.201,1,3,55,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0


### Removing Variables

As previously mentioned, we had to get rid of certain variables that didn't have any predictive power for our machine learning models. We can start by removing excess dummy variables created by pandas when we initially called for them.

Let's split our data into test set and train set and remove the excess dummy variables from the train set. That way they won't be used during testing.

In [8]:
x_train, x_test, y_train, y_test = train_test_split(bank_model_df.drop(['deposit'] ,axis=1), 
                                                    bank_model_df['deposit'],
                                                    train_size=0.7, # 70-30 split
                                                    random_state=42) # constant seed allows for reproducability
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [9]:
columns_to_drop = ['client_was_contacted_1', 'poutcome_nonexistent', 'month_apr', 'contact_cellular', 'job_self-employed', 'default_unknown']
logistic_model = sm.Logit(y_train, sm.add_constant(x_train.drop(columns_to_drop, axis=1))).fit(maxiter=1000)
print(logistic_model.summary())

         Current function value: 0.202419
         Iterations: 1000
                           Logit Regression Results                           
Dep. Variable:                deposit   No. Observations:                28831
Model:                          Logit   Df Residuals:                    28795
Method:                           MLE   Df Model:                           35
Date:                Tue, 09 Feb 2021   Pseudo R-squ.:                  0.4210
Time:                        19:33:16   Log-Likelihood:                -5836.0
converged:                      False   LL-Null:                       -10079.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   -288.6847     46.823     -6.165      0.000    -380.455    -196.914
duration                   

/Applications/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


--

The excess dummy variables have now been removed from the dataset and we can really begin to narrow down which variables are unneccesary or detrimental to our predictive power.

As we eliminate these undesirable variables, we can also simultaneously build our Logistic Regression model.

First let's check for variables that hinder our independent coefficients through multi-collinearity. We can achieve this through the use of the variance inflation factor:

In [10]:
variance_inflation_list = []
variables = pd.Series(x_train.drop(columns_to_drop, axis=1).columns)
completed_cols = []
variance_inflation_factors = []

for variable in variables:
    completed_cols.append(variable)
    dependent_variable = variable
    independent_variables = variables[~variables.isin(completed_cols)]
    mod = sm.OLS(x_train.drop(columns_to_drop, axis=1)[dependent_variable], sm.add_constant(x_train.drop(columns_to_drop, axis=1)[independent_variables.to_list()]))
    residuals = mod.fit()
    variance_inflation_factor = 1 / (1 - residuals.rsquared)
    variance_inflation_factors.append({'dependent_variable': dependent_variable, 'variance_inflation_factor': variance_inflation_factor})
    
print(pd.DataFrame(variance_inflation_factors))

        dependent_variable  variance_inflation_factor
0                 duration                   1.011601
1              nr_employed                 199.199988
2                euribor3m                 104.090973
3             emp_var_rate                   7.440831
4            cons_conf_idx                   2.220549
5           cons_price_idx                   2.621768
6                    pdays                   3.963402
7                      age                   1.405326
8                 previous                   5.568172
9   client_was_contacted_0                  11.033087
10        poutcome_failure                   1.157539
11        poutcome_success                   1.073545
12               month_aug                   2.925225
13               month_dec                   1.024404
14               month_jul                   1.522485
15               month_jun                   1.601930
16               month_mar                   1.021689
17               month_may  

--

Now that we have access to the variance inflation factors for each variable, we should uncover the C-Statistic (AUC) which will tell us how accurate the classifier is at predicting the target variable.

--

In [11]:
def df_crossjoin(df1, df2, **kwargs):
    df1['temp_key'] = 1
    df2['temp_key'] = 1
    return_df = pd.merge(df1, df2, on='temp_key', **kwargs).drop('temp_key', axis=1)
    return_df.index = pd.MultiIndex.from_product((df1.index, df2.index))
    return return_df

y_prediction = pd.DataFrame(logistic_model.predict(sm.add_constant(x_train.drop(columns_to_drop, axis=1))))
y_prediction.columns = ["probabilities"]
both_df = pd.concat([y_train, y_prediction], axis=1)
zeros_df = both_df[['deposit', 'probabilities']][both_df['deposit'] == 0]
ones_df = both_df[['deposit', 'probabilities']][both_df['deposit'] == 1]
joined_df = df_crossjoin(ones_df, zeros_df)
joined_df['concordant_pair'] = 0
joined_df.loc[joined_df['probabilities_x'] > joined_df['probabilities_y'], 'concordant_pair'] = 1
joined_df['discordant_pair'] = 0
joined_df.loc[joined_df['probabilities_x'] < joined_df['probabilities_y'], 'discordant_pair'] = 1
joined_df['tied_pair'] = 0
joined_df.loc[joined_df['probabilities_x'] == joined_df['probabilities_y'], 'tied_pair'] = 1
p_concordant = (sum(joined_df['concordant_pair']) * 1.0 ) / (joined_df.shape[0])
p_discordant = (sum(joined_df['discordant_pair']) * 1.0 ) / (joined_df.shape[0])
c_statistic = 0.5 + (p_concordant - p_discordant) / 2.0
print("C-statistic: " + str(c_statistic))

C-statistic: 0.9385172268269756


Our C-statistic is 0.9 which is > 0.7

## Evaluation of Data

### Data Verification

### Data Statistics

### Data Attributes

### Feature Selection

## Logistic Regression Model

## Random Forest Model

## Comparison of Models

## Performance Evaluation

## Results

## Deployment